# CSC440-Data Mining Homework 4(Written by Haoshu Qin)

Due: 02/26/2023

Description: Market Basket Analysis (Chapter 6) is commonly used in "recommender" systems. The basic idea is to discover interesting rules of the form {If someone likes these} -> {then they may also like these}. Download and get to know the Anonymous Microsoft Web Data Data Set: https://archive.ics.uci.edu/ml/datasets/Anonymous+Microsoft+Web+Data

All students must perform MBA using the Apriori algorithm. Students enrolled in 440 (Grad students) must in addition also perform analysis using the FP_GROW algorithm and compare the results of the two algorithms.

You may use "library" functions. The Kagle tutorial is a good resource, but you must apply this to the dataset at hand (https://www.kaggle.com/code/rockystats/apriori-algorithm-or-market-basket-analysis). Submit a pdf "paper"/"report" describing the problem, your approach to pre-processing, the tools you used, the problems you encountered and your results. Briefly discuss why your results are "interesting" and not "trivial". Discuss your choice of min_support and confidence. (The target size of the paper is 5 pages, but your mileage may vary.) Only include essential code in the body your paper. Include the full code and sample runs you used as an appendix to the paper.

Proper formatting of the paper is essential and counts for the grade. Get familiar with standard ACM (Association for Computing Machinery) standards. Please understand : results are of course important, but proper presentation is also important. For extra credit (30%) implement the algorithms yourself without libraries.

## 1. Read data and process the data

In [1]:
# process data
with open('/Users/haydee_mac/Desktop/CSC440-Data Mining/DM HW3/anonymous-msweb.data') as f:
	data = f.readlines()

# Remove the first 7 lines of the data
data = data[7:]

# Split each line of the data into a list, remove the newline character, and store the result in a list of lists
data = [i.rstrip('\n').split(',') for i in data]

# Create a list of rows in `data` where the first element of each row is 'A'
att = [i for  i in data if i[0]=='A']

# Create a list of rows in `data` where the first element of each row is 'C' or 'V'
acts = [i for i in data if i[0]=='C' or i[0]=='V']

# Create a list of indices in `acts` where the first element of each row is 'C'
idx = [i for i in range(len(acts)) if acts[i][0]=='C']

# Add one more element to `idx` to represent the end of `acts`
idx.append(len(acts)+1)

# Generate a dataset for algorithms
dataset = []
for i in range(len(idx)-1):
	# Get a slice of `acts` from the current index to the next one
	v = acts[idx[i]+1: idx[i+1]]
	
	# Extract the second element from each row of the slice and add it to the `dataset`
	dataset.append([i[1] for i in v])

# Map the names to the values in the `att` list
att_dic  = {i[1]:i[3] for i in att}

# Replace the names in the `dataset` with the corresponding values from `att_dic`
dataset = [[att_dic[i] for i in j] for j in dataset]


## 2. Apriori

Decription:

The code implements the Apriori algorithm for finding frequent itemsets in a transaction data set.

The main function of the code is the generate_L function, which generates all frequent itemsets with a maximum item number of k and a minimum support of min_support.

The generate_L function first generates the frequent 1-itemset L1 by calling create_C1 and generate_Lk_by_Ck functions. The create_C1 function creates the frequent candidate 1-itemset C1 by scanning the transaction data set. The generate_Lk_by_Ck function generates Lk from Ck by executing a delete policy.

The generate_L function then iteratively generates L2, L3, ..., Lk by calling create_Ck and generate_Lk_by_Ck functions. The create_Ck function creates Ck from Lk-1 by its own connection operation.

The code also implements the pruning step of the Apriori algorithm. The is_apriori function judges whether a frequent candidate k-itemset satisfies the Apriori property, which is that all of its (k-1)-subsets must also be frequent. If a candidate k-itemset does not satisfy the Apriori property, it will be pruned.

The support_data dictionary is used to store the support of each frequent itemset. The key of the dictionary is the frequent itemset, and the value is the support. The support_data dictionary is updated in the generate_Lk_by_Ck function.

In [2]:

def create_C1(data_set):
	"""
	Create frequent candidate 1-itemset C1 by scaning data set.
	Args:
		data_set: A list of transactions. Each transaction contains several items.
	Returns:
		C1: A set which contains all frequent candidate 1-itemsets
	"""
	C1 = set()
	for t in data_set:
		for item in t:
			item_set = frozenset([item])
			C1.add(item_set)
	return C1


def is_apriori(Ck_item, Lksub1):
	"""
	Judge whether a frequent candidate k-itemset satisfy Apriori property.
	Args:
		Ck_item: a frequent candidate k-itemset in Ck which contains all frequent
				 candidate k-itemsets.
		Lksub1: Lk-1, a set which contains all frequent candidate (k-1)-itemsets.
	Returns:
		True: satisfying Apriori property.
		False: Not satisfying Apriori property.
	"""
	for item in Ck_item:
		sub_Ck = Ck_item - frozenset([item])
		if sub_Ck not in Lksub1:
			return False
	return True


def create_Ck(Lksub1, k):
	"""
	Create Ck, a set which contains all all frequent candidate k-itemsets
	by Lk-1's own connection operation.
	Args:
		Lksub1: Lk-1, a set which contains all frequent candidate (k-1)-itemsets.
		k: the item number of a frequent itemset.
	Return:
		Ck: a set which contains all all frequent candidate k-itemsets.
	"""
	Ck = set()
	len_Lksub1 = len(Lksub1)
	list_Lksub1 = list(Lksub1)
	for i in range(len_Lksub1):
		for j in range(1, len_Lksub1):
			l1 = list(list_Lksub1[i])
			l2 = list(list_Lksub1[j])
			l1.sort()
			l2.sort()
			if l1[0:k-2] == l2[0:k-2]:
				Ck_item = list_Lksub1[i] | list_Lksub1[j]
				# pruning
				if is_apriori(Ck_item, Lksub1):
					Ck.add(Ck_item)
	return Ck


def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
	"""
	Generate Lk by executing a delete policy from Ck.
	Args:
		data_set: A list of transactions. Each transaction contains several items.
		Ck: A set which contains all all frequent candidate k-itemsets.
		min_support: The minimum support.
		support_data: A dictionary. The key is frequent itemset and the value is support.
	Returns:
		Lk: A set which contains all all frequent k-itemsets.
	"""
	Lk = set()
	item_count = {}
	for t in data_set:
		for item in Ck:
			if item.issubset(t):
				if item not in item_count:
					item_count[item] = 1
				else:
					item_count[item] += 1
	t_num = float(len(data_set))
	for item in item_count:
		if (item_count[item] / t_num) >= min_support:
			Lk.add(item)
			support_data[item] = item_count[item] / t_num
	return Lk


def generate_L(data_set, k, min_support):
	"""
	Generate all frequent itemsets.
	Args:
		data_set: A list of transactions. Each transaction contains several items.
		k: Maximum number of items for all frequent itemsets.
		min_support: The minimum support.
	Returns:
		L: The list of Lk.
		support_data: A dictionary. The key is frequent itemset and the value is support.
	"""
	support_data = {}
	C1 = create_C1(data_set)
	L1 = generate_Lk_by_Ck(data_set, C1, min_support, support_data)
	Lksub1 = L1.copy()
	L = []
	L.append(Lksub1)
	for i in range(2, k+1):
		Ci = create_Ck(Lksub1, i)
		Li = generate_Lk_by_Ck(data_set, Ci, min_support, support_data)
		Lksub1 = Li.copy()
		L.append(Lksub1)
	return L, support_data


def generate_big_rules(L, support_data, min_conf):
	"""
	Generate big rules from frequent itemsets.
	Args:
		L: The list of Lk.
		support_data: A dictionary. The key is frequent itemset and the value is support.
		min_conf: Minimal confidence.
	Returns:
		big_rule_list: A list which contains all big rules. Each big rule is represented
					   as a 3-tuple.
	"""
	big_rule_list = []
	sub_set_list = []
	for i in range(0, len(L)):
		for freq_set in L[i]:
			for sub_set in sub_set_list:
				if sub_set.issubset(freq_set):
					sup_AB = support_data[freq_set]
					sup_B = support_data[sub_set]
					conf = support_data[freq_set] / support_data[freq_set - sub_set]
					lift = conf / sup_B
					big_rule = (freq_set - sub_set, sub_set,sup_AB, conf,lift)
					if conf >= min_conf and big_rule not in big_rule_list:
						big_rule_list.append(big_rule)
			sub_set_list.append(freq_set)
	return big_rule_list

# main
def apriori(data_set,minSup=0.2, minConf=0.1):
	print(f"Params：\nmin support: {minSup}\nmin confidence: {minConf}")
	L, support_data = generate_L(data_set, k=2, min_support=minSup)
	big_rules_list = generate_big_rules(L, support_data, min_conf=minConf)
	freqItemSet = []
	for Lk in L:
		for freq_set in Lk:
			freqItemSet.append([freq_set,support_data[freq_set]])
	return big_rules_list


rules1 = apriori(dataset,minSup=0.01, minConf=0.5)

Params：
min support: 0.01
min confidence: 0.5


In [3]:
for a,b,c,d,e in rules1:
	print(f"{a} ==> {b}, support={c:.4f}, confidence={c:.4f}")

frozenset({'"Windows 95"'}) ==> frozenset({'"Windows Family of OSs"'}), support=0.0324, confidence=0.0324
frozenset({'"Internet Explorer"'}) ==> frozenset({'"Free Downloads"'}), support=0.1608, confidence=0.1608
frozenset({'"Windows95 Support"'}) ==> frozenset({'"Windows Family of OSs"'}), support=0.0328, confidence=0.0328
frozenset({'"Office Free Stuff"'}) ==> frozenset({'"MS Office Info"'}), support=0.0116, confidence=0.0116
frozenset({'"Windows NT Workstation"'}) ==> frozenset({'"Windows Family of OSs"'}), support=0.0142, confidence=0.0142
frozenset({'"NetMeeting"'}) ==> frozenset({'"Free Downloads"'}), support=0.0112, confidence=0.0112
frozenset({'"Web Site Builder\'s Gallery"'}) ==> frozenset({'"Internet Site Construction for Developers"'}), support=0.0353, confidence=0.0353
frozenset({'"IE Support"'}) ==> frozenset({'"Internet Explorer"'}), support=0.0137, confidence=0.0137
frozenset({'"Windows 95"'}) ==> frozenset({'"Products "'}), support=0.0189, confidence=0.0189
frozenset({'"

## 3. Fp-Grow

Description:

This is an implementation of the FP-growth algorithm in Python. The FP-growth algorithm is used for frequent itemset mining and association rule learning. The implementation is based on a tree-based data structure called an FP-tree. The goal of the algorithm is to find frequent item sets (sets of items that occur together frequently in a dataset) from a transaction database, where a transaction is a set of items. The algorithm first builds an FP-tree from the transaction database and then generates frequent itemsets from the FP-tree by using a pattern growth approach.

The code defines two classes, Node and Fp_growth. The Node class is used to define the structure of each node in the FP-tree, with each node having a name, a count (the number of times the item represented by the node appears in the transaction database), a nodeLink (a link to the next node with the same item name), a parent node, and a dictionary of children nodes.

The Fp_growth class implements the FP-growth algorithm, with the main function being create_fptree. This function takes as input a dictionary of transactions, the minimum support (the minimum number of times an item must appear in the transaction database to be considered frequent), and a flag that indicates whether the output should be in the form of rules or not. The function first builds the header table and the frequent item set. It then builds the FP-tree by updating the tree based on the transactions in the database. Finally, it generates frequent item sets by using a recursive approach to find frequent patterns in the tree.

In [4]:
class Node:
	def __init__(self, node_name, count, parentNode):
		self.name = node_name # node name
		self.count = count # count of node name
		self.nodeLink = None # used to find all nodes with the same node name in the whole tree
		self.parent = parentNode # parent node
		self.children = {} # children nodes, {node name: node address}

class Fp_growth:
	def data_compress(self, data_set):
		data_dic = {}
		for i in data_set:
			if frozenset(i) not in data_dic:
				data_dic[frozenset(i)] = 1
			else:
				data_dic[frozenset(i)] += 1
		return data_dic

	def update_header(self, node, targetNode): # update the linked list in headertable
		while node.nodeLink != None:
			node = node.nodeLink
		node.nodeLink = targetNode

	def update_fptree(self, items, count, node, headerTable): # used to update fptree
		if items[0] in node.children:
			# check if the first node in items has already been a child node
			node.children[items[0]].count += count
		else:
			# create a new branch
			node.children[items[0]] = Node(items[0], count, node)
			# update the linked list of the corresponding frequent itemset, add it afterwards
			if headerTable[items[0]][1] == None:
				headerTable[items[0]][1] = node.children[items[0]]
			else:
				self.update_header(headerTable[items[0]][1], node.children[items[0]])
		# recursive
		if len(items) > 1:
			self.update_fptree(items[1:], count, node.children[items[0]], headerTable)

	def create_fptree(self, data_dic, min_support):  # Main function for creating tree
		item_count = {}  # Count the number of occurrences of each item
		for t in data_dic:  # First traverse to get frequent one-item sets
			for item in t:
				if item not in item_count:
					item_count[item] = data_dic[t]
				else:
					item_count[item] += data_dic[t]
		headerTable = {}
		for k in item_count:  # Remove items that do not meet the minimum support
			if item_count[k] >= min_support:
				headerTable[k] = item_count[k]

		freqItemSet = set(headerTable.keys())  # The frequent item set that meets the minimum support
		if len(freqItemSet) == 0:
			return None, None
		for k in headerTable:
			headerTable[k] = [headerTable[k], None]  # element: [count, node]
		tree_header = Node('head node', 1, None)
		ite = data_dic
		for t in ite:  # Second traverse, build tree
			localD = {}
			for item in t:
				if item in freqItemSet:  # Filter, only take the frequent items that meet the minimum support in this sample
					localD[item] = headerTable[item][0]  # element : count
			if len(localD) > 0:
				# Sort the single sample based on global frequency from large to small
				order_item = [v[0] for v in sorted(localD.items(), key=lambda x: x[1], reverse=True)]
				# Update the tree with the filtered and sorted sample
				self.update_fptree(order_item, data_dic[t], tree_header, headerTable)
		return tree_header, headerTable

	def find_path(self, node, nodepath):
		'''
		Recursively add the parent node of node to the path
		'''
		if node.parent != None:
			nodepath.append(node.parent.name)
			self.find_path(node.parent, nodepath)

	def find_cond_pattern_base(self, node_name, headerTable):
		'''
		Find all conditional pattern bases based on the node name
		'''
		treeNode = headerTable[node_name][1]
		cond_pat_base = {}  # Save all conditional pattern bases
		while treeNode != None:
			nodepath = []
			self.find_path(treeNode, nodepath)
			if len(nodepath) > 1:
				cond_pat_base[frozenset(nodepath[:-1])] = treeNode.count
			treeNode = treeNode.nodeLink
		return cond_pat_base


	def create_cond_fptree(self, headerTable, min_support, temp, freq_items, support_data):
		# Initial frequent items are elements in headerTable
		freqs = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]  # sort based on the total frequency of frequent items
		for freq in freqs:  # for each frequent item
			freq_set = temp.copy()
			freq_set.add(freq)
			freq_items.add(frozenset(freq_set))
			if frozenset(freq_set) not in support_data:  # check if the frequent item is in support_data
				support_data[frozenset(freq_set)] = headerTable[freq][0]
			else:
				support_data[frozenset(freq_set)] += headerTable[freq][0]

			cond_pat_base = self.find_cond_pattern_base(freq, headerTable)  # find all conditional pattern bases
			# create conditional pattern tree
			cond_tree, cur_headtable = self.create_fptree(cond_pat_base, min_support)
			if cur_headtable != None:
				self.create_cond_fptree(cur_headtable, min_support, freq_set, freq_items, support_data)  # recursive mine on the conditional FP tree

	def generate_L(self, data_set, min_support):
		# compress the data set into a dictionary with frequency count
		data_dic = self.data_compress(data_set)

		# initialize the frequent item set and support data
		freqItemSet = set()
		support_data = {}

		# create the fptree of the dataset and get the header table
		tree_header, headerTable = self.create_fptree(data_dic, min_support)

		# create fptree of each frequent one-item and mine frequent items and save support count
		self.create_cond_fptree(headerTable, min_support, set(), freqItemSet, support_data)

		# find the maximum length of frequent items
		max_l = 0
		for i in freqItemSet:
			if len(i) > max_l:
				max_l = len(i)

		# create a container L to store the frequent items based on their length
		L = [set() for _ in range(max_l)]
		for i in freqItemSet:
			L[len(i) - 1].add(i)
		
		return L, support_data

	def generate_R(self, data_set, min_support, min_conf):
		# generate L, support_data using the generate_L method
		L, support_data = self.generate_L(data_set, min_support)
		
		# calculate the number of transactions in the data set
		dataset_length = len(data_set)
		
		# initialize the list to store association rules
		rule_list = []
		
		# initialize the list to store subsets of frequent itemsets
		sub_set_list = []
		
		# loop through each frequent itemset in L
		for i in range(0, len(L)):
			for freq_set in L[i]:
				
				# loop through each subset in sub_set_list
				for sub_set in sub_set_list:
					
					# if the subset is a proper subset of the frequent itemset,
					# and the difference between the frequent itemset and the subset is in support_data
					if sub_set.issubset(freq_set) and freq_set - sub_set in support_data:
						
						# calculate the support of A
						supp_A = support_data[freq_set - sub_set] 
						
						# calculate the confidence of the association rule (A -> B)
						conf = support_data[freq_set] / supp_A   
						
						# calculate the support of A & B
						supp_AB = support_data[freq_set]/dataset_length  
						
						# calculate the support of B
						supp_B = (support_data[sub_set]/dataset_length)
						
						# calculate the lift of the association rule (A -> B)
						lift = conf / supp_B
						
						# create a tuple to store the association rule (A, B, support of A & B, confidence, lift)
						big_rule = (freq_set - sub_set, sub_set, supp_AB, conf, lift)
						
						# if the confidence is greater than or equal to min_conf and the association rule is not in rule_list, add the rule to rule_list
						if conf >= min_conf and big_rule not in rule_list:
							rule_list.append(big_rule)
							
				# add the frequent itemset to sub_set_list
				sub_set_list.append(freq_set)
		
		# sort the association rules in rule_list by support of A & B in descending order
		rule_list = sorted(rule_list, key=lambda x: (x[2]), reverse=True)
		
		# return the list of association rules
		return rule_list


def load_data(dataset): 
	ans = [] 
	for row in dataset:
		row = list(set(row)) 
		row.sort()
		ans.append(row) 
	return ans 


def run_fpgrowth(data_set,min_support=0.1,min_conf=0.5):
	# main
	min_support = int(min_support*len(data_set))
	data_set = load_data(data_set)
	fp = Fp_growth()
	rule_list = fp.generate_R(data_set, min_support, min_conf)
	return rule_list
rules2 = run_fpgrowth(dataset,min_support=0.01,min_conf=0.5)

In [5]:
for a,b,c,d,e in rules2:
	print(f"{a} ==> {b}, support={c:.4f}, confidence={c:.4f}")

# We use file reading and list processing data to build a nested list data set for easy input into the algorithm
# Choose the minimum support degree as 0.01, the minimum confidence degree as 0.5, and use the two algorithms to mine separately. 
# It is found that under the same support degree and confidence degree, the speed of fp-grow is significantly faster than that of apriori

# We found our results to be interesting, people visiting "Internet Explorer" are more likely to visit "Free Downloads", 
# This means that people using browsers have a significant download demand
# people visiting "Knowledge Base" are more likely to visit "Support Desktop", 
# which means that people who enjoy knowledge are more likely to seek help from others

frozenset({'"Internet Explorer"'}) ==> frozenset({'"Free Downloads"'}), support=0.1608, confidence=0.1608
frozenset({'"Windows Family of OSs"'}) ==> frozenset({'"Free Downloads"'}), support=0.0779, confidence=0.0779
frozenset({'"Knowledge Base"'}) ==> frozenset({'"Support Desktop"'}), support=0.0552, confidence=0.0552
frozenset({'"Knowledge Base"'}) ==> frozenset({'"isapi"'}), support=0.0469, confidence=0.0469
frozenset({'"Windows95 Support"'}) ==> frozenset({'"isapi"'}), support=0.0461, confidence=0.0461
frozenset({'"Web Site Builder\'s Gallery"'}) ==> frozenset({'"Internet Site Construction for Developers"'}), support=0.0353, confidence=0.0353
frozenset({'"isapi"', '"Knowledge Base"'}) ==> frozenset({'"Support Desktop"'}), support=0.0332, confidence=0.0332
frozenset({'"isapi"', '"Support Desktop"'}) ==> frozenset({'"Knowledge Base"'}), support=0.0332, confidence=0.0332
frozenset({'"Knowledge Base"', '"Support Desktop"'}) ==> frozenset({'"isapi"'}), support=0.0332, confidence=0.0332
f